# Creek analysis of sediment reduce temporally

Whitebox Tools Manual: <a href='https://jblindsay.github.io/wbt_book/intro.html'>Link</a>

Developed with the Anaconda Python distribution: <a href='https://anaconda.org'> www.anaconda.org</a>

To install rasterio and whitebox, type in the terminal   
`conda install -c conda-forge whitebox`  
`conda install -c conda-forge rasterio`  

In [ ]:
##################################################################
# Import essential packages
##################################################################

import numpy as np
import matplotlib.pyplot as plt

import imageio
import rasterio
from rasterio.transform import from_origin
import time

%matplotlib inline
on=1;off=0

# Importing the Whitebox tools
import whitebox
wbt = whitebox.WhiteboxTools()

##################################################################
# Set working path
##################################################################

root_dir = '/Volumes/T7/GC_SedimentReduceTemporalQX/Root/'
data_dir = '/Users/gccheng/CreekAnalyze/SedimentReduceTemporal/Data50/'

LengthX  = 200.0  # 100.0  Size of the domain in physical dimensions
LengthY  = 200.0  # 100.0  Size of the domain in physical dimensions

# set whitebox working directory
wbt.set_working_dir(data_dir)
wbt.verbose = False

##################################################################
# Function definition
##################################################################

def PlotImage(Image,title, cmap, clim=None):
    
    if isinstance(Image, str):
        Image = imageio.imread(Image)

    fig, ax = plt.subplots(1, 1, figsize=(15, 15))

    im=ax.imshow(Image, cmap=cmap)
    ax.set_title(title, fontsize=15)
    ax.set_axis_off()
    if clim!=None: im.set_clim(clim)

    f=fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04, orientation='vertical')

def SaveAsGeoTiff(Map,Name):
    with  rasterio.open(Name, 'w', driver='GTiff',
                     height=Map.shape[0], width=Map.shape[1],
                     count=1, dtype=Map.dtype,
                     crs='+proj=latlong', transform=transform) as Im:
        Im.write(Map, 1)
        Im.close()

def SaveAsPng(input,output,cmap):
    D = imageio.imread(input)
    plt.imsave(output,(D/np.max(D)*256), cmap=cmap)

CoverThreshold=100

# def GetBasins(CurrentTime):
def GetBasins( ):

    global CoverStore, BasinNrMax
    # SaveAsGeoTiff(ss[:,:,CurrentTime],data_dir+'DEM.tif')
    SaveAsGeoTiff(ss[:,:],data_dir+'DEM.tif')

    ### Deriving the flow accumulation and pointer maps

    dem       = data_dir+"DEM.tif"
    out_dem   = data_dir+"filled_dem.tif"
    out_pntr  = data_dir+"filled_pntr.tif"
    out_accum = data_dir+"filled_accum.tif"
    out_type  = "ca"

    wbt.flow_accumulation_full_workflow(
        dem, 
        out_dem, 
        out_pntr, 
        out_accum, 
        out_type, 
        log=False, 
        clip=True, 
        esri_pntr=False);

    # Drainage Basins
    out_basins = data_dir + f'Basins.tif'

    wbt.basins(
        out_pntr, 
        out_basins, 
        esri_pntr=False)
    
    #Delineating the streams
    out_streams = data_dir + f'streams'+format(deltaS)+'_'+format(repeat)+'.tif'
    threshold = 50

    wbt.extract_streams(
        out_accum, 
        out_streams, 
        threshold, 
        zero_background=True)

    # Rereading the basin and streams Tif images
    Image = imageio.imread(out_basins)
    Streams = imageio.imread(out_streams)
    
    # Obtaining basin sizes
    BasinNrs, BasinCover = np.unique(Image.astype(int), return_counts=True)

    # Sorting according to basin size
    Ind = np.argsort(-BasinCover)
    BasinNrs = BasinNrs[Ind]
    BasinCover = BasinCover[Ind]

    # Only Basins larger then CoverThreshold are included
    Ind=BasinCover>CoverThreshold
    BasinCover=BasinCover[Ind]
    BasinNrs = BasinNrs[Ind]

    Number_of_Basins=BasinCover.shape[0]
    
    # Plotting the Streams on the image
    Image[Streams==1]=1

    return Image, BasinCover    


##################################################################
# Main part
##################################################################

NumFrames = 49 # NumFrames from 0-49 is stable, NumFrames from 50-99 is degradation
for deltaS in range(1, 10):
    deltaname = deltaS/10
    for repeat in range(1,6):
        
        filename = 'SedimentReduceTemporal_'+format(deltaname)+'_'+format(repeat)+'_AllData.npz'
        print(filename)
        
        Data = np.load(root_dir+filename)
                # us = Data['u']
        # vs = Data['v']
        # hs = Data['h']
        ss = Data['s']
        # bs = Data['b']
        # ds = Data['d']
        ss = ss[0::2,0::2,NumFrames]
        Xdim,Ydim = ss.shape
        
        # Setting up coordinates
        x = np.linspace(-LengthX/2, LengthX/2, Xdim)
        y = np.linspace(-LengthY/2, LengthY/2, Ydim)
        X, Y = np.meshgrid(x, y)
        res = (x[-1] - x[0]) / Xdim
        
        # Setting up coordinates
        x = np.linspace(-LengthX/2, LengthX/2, Xdim)
        y = np.linspace(-LengthY/2, LengthY/2, Ydim)
        X, Y = np.meshgrid(x, y)
        
        # Creating a transformation
        transform = from_origin(x[0] - res / 2, y[-1] + res / 2, res, res) 
        
        Image, BasinCover = GetBasins( )
        
        # PlotImage(Image,'Basins', cmap='prism')
        
        # Plot DEM image
        fig, ax = plt.subplots(1,1, figsize=(30, 15))
        im1=ax.imshow(ss[:,:], animated=True, interpolation='bilinear')
        ax.set_axis_off()
        plt.tight_layout()
        fig.savefig(data_dir + 'DEM'+format(deltaS)+'_'+format(repeat)+'.tif',dpi = 100, bbox_inches = 'tight', pad_inches = 0)

The End, Johan van de Koppel 2019